# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup

## Data read-in

In [2]:
df = pd.read_csv("apartmet buildings with floating rate loans - Sheet1.csv")
df

,building name,address,owner,balance,maturity date,watchlisted?,delinquent?,DSCR,Occupancy
0,The Avondale,"4200 West Belmont Avenue, Chicago",Noah Properties,$25.9M,April 2024,Yes,No,0.55,83%
1,330 South Wells Apartments,"330 South Wells Street, Chicago",LG Group,$27.3M,November 2024,Yes,No,0.76,92%
2,Fisher Building City Apartments,"343 South Dearborn Street, Chicago",City Club Apartments,$28.1M,May 2027,Yes,No,0.89,81%
3,Two East Oak,"2 East Oak Street, Chicago",Strategic Properties of North America,$20.2M,February 2025,No,No,0.72,92%
4,Lakes of Schaumburg,"801 Belinder Lane, Schaumburg",Morgan Properties,$65.1M,August 2026,No,No,0.94,94%
5,Gates of Deer Cove,"125 West Dundee Road, Palatine",Morgan Properties,$28.1M,August 2026,Yes,No,0.73,94%
6,500 Station Apartments,"500 Station Boulevard, Aurora",Amit Goel,$125M,April 2025,Yes,No,0.53,89%
7,Cityview at Highlands,"2720 South Highland Ave, Lombard",Torchlight Investors,$73.5M,January 2032,No,No,0.87,93%
8,100 North La Salle,"100 North La Salle Street, Chicago",Farbman Group,$13.5M,September 2024,Yes,No,NaN,80%
9,1334 North Dearborn,"1334 North Dearborn, Chicago",Strategic Properties of North America,$21.6M,July 2025,Yes,No,0.65,95%


## Geocode

In [3]:
df['full_address'] = df['address'] + ' IL'

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,building name,address,owner,balance,maturity date,watchlisted?,delinquent?,DSCR,Occupancy,full_address,geocoded,lat,lon,geometry
0,The Avondale,"4200 West Belmont Avenue, Chicago",Noah Properties,$25.9M,April 2024,Yes,No,0.55,83%,"4200 West Belmont Avenue, Chicago IL","(41.9392528, -87.73226760000001)",41.939253,-87.732268,POINT (-87.73227 41.93925)
1,330 South Wells Apartments,"330 South Wells Street, Chicago",LG Group,$27.3M,November 2024,Yes,No,0.76,92%,"330 South Wells Street, Chicago IL","(41.8771327, -87.6338681)",41.877133,-87.633868,POINT (-87.63387 41.87713)
2,Fisher Building City Apartments,"343 South Dearborn Street, Chicago",City Club Apartments,$28.1M,May 2027,Yes,No,0.89,81%,"343 South Dearborn Street, Chicago IL","(41.8773196, -87.62909619999999)",41.877320,-87.629096,POINT (-87.62910 41.87732)
3,Two East Oak,"2 East Oak Street, Chicago",Strategic Properties of North America,$20.2M,February 2025,No,No,0.72,92%,"2 East Oak Street, Chicago IL","(41.9009321, -87.6280857)",41.900932,-87.628086,POINT (-87.62809 41.90093)
4,Lakes of Schaumburg,"801 Belinder Lane, Schaumburg",Morgan Properties,$65.1M,August 2026,No,No,0.94,94%,"801 Belinder Lane, Schaumburg IL","(42.0760418, -88.0589234)",42.076042,-88.058923,POINT (-88.05892 42.07604)
5,Gates of Deer Cove,"125 West Dundee Road, Palatine",Morgan Properties,$28.1M,August 2026,Yes,No,0.73,94%,"125 West Dundee Road, Palatine IL","(42.1388241, -88.0463388)",42.138824,-88.046339,POINT (-88.04634 42.13882)
6,500 Station Apartments,"500 Station Boulevard, Aurora",Amit Goel,$125M,April 2025,Yes,No,0.53,89%,"500 Station Boulevard, Aurora IL","(41.7640731, -88.2145728)",41.764073,-88.214573,POINT (-88.21457 41.76407)
7,Cityview at Highlands,"2720 South Highland Ave, Lombard",Torchlight Investors,$73.5M,January 2032,No,No,0.87,93%,"2720 South Highland Ave, Lombard IL","(41.8385751, -88.0152428)",41.838575,-88.015243,POINT (-88.01524 41.83858)
8,100 North La Salle,"100 North La Salle Street, Chicago",Farbman Group,$13.5M,September 2024,Yes,No,NaN,80%,"100 North La Salle Street, Chicago IL","(41.9008285, -87.6326259)",41.900829,-87.632626,POINT (-87.63263 41.90083)
9,1334 North Dearborn,"1334 North Dearborn, Chicago",Strategic Properties of North America,$21.6M,July 2025,Yes,No,0.65,95%,"1334 North Dearborn, Chicago IL","(41.9068227, -87.6304007)",41.906823,-87.630401,POINT (-87.63040 41.90682)


In [6]:
gdf

,building name,address,owner,balance,maturity date,watchlisted?,delinquent?,DSCR,Occupancy,full_address,geocoded,lat,lon,geometry
0,The Avondale,"4200 West Belmont Avenue, Chicago",Noah Properties,$25.9M,April 2024,Yes,No,0.55,83%,"4200 West Belmont Avenue, Chicago IL","(41.9392528, -87.73226760000001)",41.939253,-87.732268,POINT (-87.73227 41.93925)
1,330 South Wells Apartments,"330 South Wells Street, Chicago",LG Group,$27.3M,November 2024,Yes,No,0.76,92%,"330 South Wells Street, Chicago IL","(41.8771327, -87.6338681)",41.877133,-87.633868,POINT (-87.63387 41.87713)
2,Fisher Building City Apartments,"343 South Dearborn Street, Chicago",City Club Apartments,$28.1M,May 2027,Yes,No,0.89,81%,"343 South Dearborn Street, Chicago IL","(41.8773196, -87.62909619999999)",41.877320,-87.629096,POINT (-87.62910 41.87732)
3,Two East Oak,"2 East Oak Street, Chicago",Strategic Properties of North America,$20.2M,February 2025,No,No,0.72,92%,"2 East Oak Street, Chicago IL","(41.9009321, -87.6280857)",41.900932,-87.628086,POINT (-87.62809 41.90093)
4,Lakes of Schaumburg,"801 Belinder Lane, Schaumburg",Morgan Properties,$65.1M,August 2026,No,No,0.94,94%,"801 Belinder Lane, Schaumburg IL","(42.0760418, -88.0589234)",42.076042,-88.058923,POINT (-88.05892 42.07604)
5,Gates of Deer Cove,"125 West Dundee Road, Palatine",Morgan Properties,$28.1M,August 2026,Yes,No,0.73,94%,"125 West Dundee Road, Palatine IL","(42.1388241, -88.0463388)",42.138824,-88.046339,POINT (-88.04634 42.13882)
6,500 Station Apartments,"500 Station Boulevard, Aurora",Amit Goel,$125M,April 2025,Yes,No,0.53,89%,"500 Station Boulevard, Aurora IL","(41.7640731, -88.2145728)",41.764073,-88.214573,POINT (-88.21457 41.76407)
7,Cityview at Highlands,"2720 South Highland Ave, Lombard",Torchlight Investors,$73.5M,January 2032,No,No,0.87,93%,"2720 South Highland Ave, Lombard IL","(41.8385751, -88.0152428)",41.838575,-88.015243,POINT (-88.01524 41.83858)
8,100 North La Salle,"100 North La Salle Street, Chicago",Farbman Group,$13.5M,September 2024,Yes,No,NaN,80%,"100 North La Salle Street, Chicago IL","(41.9008285, -87.6326259)",41.900829,-87.632626,POINT (-87.63263 41.90083)
9,1334 North Dearborn,"1334 North Dearborn, Chicago",Strategic Properties of North America,$21.6M,July 2025,Yes,No,0.65,95%,"1334 North Dearborn, Chicago IL","(41.9068227, -87.6304007)",41.906823,-87.630401,POINT (-87.63040 41.90682)


In [7]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [8]:
gdf.to_file('apartment_floating_rate.geojson', driver='GeoJSON')

In [9]:
gdf.columns

Index(['building name', 'address', 'owner', 'balance', 'maturity date',
       'watchlisted?', 'delinquent?', 'DSCR', 'Occupancy', 'full_address',
       'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')

In [10]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/apartment_buildings_floating_rate_loans
